### Привет!
 Перед началом выполнения кода проекта необходимо установить зависмости прописанные  в ячейке ниже

In [ ]:
!pip install langchain langchain-openai langchain-community  pydantic   tiktoken requests langchain_text_splitters  langchain_core typing

Здесь необходимо будет прописать свой API

In [ ]:
API = "YOUR_API_KEY_HERE"
URL = "https://openrouter.ai/api/v1"
giga = "deepseek/deepseek-r1-0528:free"

In [ ]:
from typing import List, Optional, Dict
from pydantic import BaseModel, Field
import json

from langchain_openai import ChatOpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.messages import SystemMessage, HumanMessage

import re

In [ ]:
class CandidateProfile(BaseModel):
    name: str
    position: str
    grade: str
    experience: str

class ExtractedProfile(BaseModel):
    position: str
    estimated_grade: str
    experience_summary: str
    confidence: int

class TurnLog(BaseModel):
    turn_id: int
    agent_visible_message: str
    user_message: str
    internal_thoughts: Optional[str] = None

class InterviewState(BaseModel):
    candidate_profile: CandidateProfile
    next_action: Optional[str] = None
    interviewer_notes: Optional[str] = None
    current_topic: Optional[str] = None
    turn_id: int = 1
    dialogue_history: List[TurnLog] = Field(default_factory=list)
    last_user_message: Optional[str] = None
    agent_visible_message: Optional[str] = None
    covered_topics: List[str] = Field(default_factory=list)
    last_topic: Optional[str] = None
    difficulty_level: str = "easy"
    stop_interview: bool = False
    red_flag: bool = False
    observer_evaluation: Optional[str] = None
    observer_notes: Optional[str] = None
    observer_instruction: Optional[str] = None
    difficulty_adjustment: Optional[str] = None


In [ ]:

profile_llm  = ChatOpenAI(base_url=URL,model=giga, api_key=API, temperature=0)
observer_llm = ChatOpenAI(base_url=URL,model=giga, api_key=API, temperature=0)
interviewer_llm = ChatOpenAI(base_url=URL,model=giga, api_key=API, temperature=0.3)
hiring_llm = ChatOpenAI(base_url=URL,model=giga, api_key=API, temperature=0)

PROFILE_ANALYZER_PROMPT = """
Ты — HR-ассистент, анализирующий самопрезентацию кандидата.

Отвечай ТОЛЬКО на русском языке.

Задача:
- Определи целевую позицию
- Оцени уровень (Junior / Middle / Senior)
- Кратко опиши опыт (1–2 предложения)
- Оцени уверенность (0–100)

Правила:
- Используй ТОЛЬКО информацию из текста
- Ничего не выдумывай
- Если сомневаешься — выбирай более низкий уровень
- Верни ТОЛЬКО валидный JSON:

{
  "position": "string",
  "estimated_grade": "string",
  "experience_summary": "string",
  "confidence": number
}
"""
OBSERVER_SYSTEM_PROMPT = """
Ты — старший инженер, наблюдающий за техническим интервью.

Общайся ТОЛЬКО на русском языке.

Ты НИКОГДА не общаешься с кандидатом.

Ты ОБЯЗАН отвечать ТОЛЬКО валидным JSON.
Никакого текста вне JSON.

Формат:

{
  "internal_thoughts": "string (на русском)",
  "evaluation": "strong | ok | weak | hallucination",
  "instruction": "ask_followup | simplify | challenge | proceed | stop | answer_candidate"
}

Правила:

- Если кандидат задал вопрос → instruction = "answer_candidate"
- Если обнаружена ложь / выдумка → evaluation = "hallucination", instruction = "challenge"
- Если ответ слабый → "challenge"
- Если хороший → "proceed"
- Если диалог зашёл в тупик → "stop"
"""

INTERVIEWER_SYSTEM_PROMPT = """
Ты — технический интервьюер.

Ты общаешься с кандидатом ТОЛЬКО на русском языке.

Правила:
- Задавай ОДИН вопрос за раз
- Будь вежливым и профессиональным
- Никогда явно не оценивай кандидата
- Следуй инструкциям наблюдателя
- Не используй английский язык
"""
HIRING_MANAGER_PROMPT = """
Ты — опытный hiring manager.

Ты анализируешь техническое интервью.

Отвечай ТОЛЬКО на русском языке.
Верни ТОЛЬКО валидный JSON.

Твоя задача — дать ЧЕСТНУЮ и ДЕТАЛЬНУЮ оценку.

Обязательно:

1. Используй оценки observer (evaluation).
2. Учитывай случаи hallucination и ухода от темы.
3. Не оставляй разделы пустыми.
4. Делай выводы на основе диалога.

Если кандидат:
- даёт ложную информацию → снижай оценку
- уходит от вопросов → снижать soft skills
- даёт хорошие ответы → фиксировать как skill

Формат:

{
  "decision": {
    "grade": "Junior | Middle | Senior",
    "hiring_recommendation": "Hire | No Hire | Strong Hire",
    "confidence_score": number
  },

  "hard_skills": {
    "confirmed_skills": [
      {
        "topic": "string",
        "evidence": "string"
      }
    ],

    "knowledge_gaps": [
      {
        "topic": "string",
        "candidate_answer": "string",
        "correct_answer": "string"
      }
    ]
  },

  "soft_skills": {
    "clarity": "low | medium | high",
    "honesty": "low | medium | high",
    "engagement": "low | medium | high"
  },

  "roadmap": [
    "string"
  ]
}

Никакого markdown.
Никаких пояснений.
"""


In [ ]:
PROFILE_SCHEMA = {
    "position": str,             # позиция или роль (Analyst, Developer, CTO)
    "estimated_grade": str,      # любая строка (Junior, Senior, CTO, Intern)
    "experience_summary": str,   # текстовое описание опыта
    "confidence": int            # 0-100
}

In [ ]:
def normalize_profile_output(data: dict) -> dict:
    normalized = {}

    # 1. позиция
    for key in ["position", "target_position", "role", "desired_position"]:
        if key in data:
            normalized["position"] = data[key]
            break
    else:
        normalized["position"] = "Unknown"

    # 2. грейд
    grade = data.get("estimated_grade") or data.get("grade") or data.get("seniority") or "Unknown"
    normalized["estimated_grade"] = str(grade)

    # 3. опыт
    normalized["experience_summary"] = data.get(
        "experience_summary"
    ) or data.get("experience") or data.get("background") or "Not provided"

    # 4. confidence
    conf = data.get("confidence", 50)
    if isinstance(conf, dict):
        conf = min(conf.values())
    elif isinstance(conf, list):
        conf = min(conf)
    elif not isinstance(conf, int):
        conf = 50

    normalized["confidence"] = max(0, min(100, int(conf)))

    return normalized

In [ ]:
def validate_profile(data: dict) -> bool:
    return all(
        k in data and (data[k] is not None) and (data[k] != "")
        for k in PROFILE_SCHEMA
    )

In [ ]:
def extract_candidate_profile(intro_text: str, llm) -> ExtractedProfile:
    """
    Извлекает профиль кандидата из вступительного текста
    """
    try:
        response = llm.invoke([
            SystemMessage(content=PROFILE_ANALYZER_PROMPT),
            HumanMessage(content=intro_text)
        ])

        # Убираем возможные markdown блоки и пробелы
        raw = response.content.strip()
        raw = re.sub(r"```json|```", "", raw).strip()

        if not raw:
            raise ValueError("Empty response from LLM")

        parsed = json.loads(raw)
        normalized = normalize_profile_output(parsed)

        if not validate_profile(normalized):
            raise ValueError("Profile validation failed")

        return ExtractedProfile(**normalized)

    except json.JSONDecodeError as e:
        print(f"[ProfileAnalyzer ERROR] JSON parsing failed: {e}")
        print(f"Raw output: {raw if 'raw' in locals() else 'No response'}")

        # Fallback: попробуем извлечь данные через регулярные выражения
        return extract_profile_fallback(intro_text)
    except Exception as e:
        print(f"[ProfileAnalyzer ERROR] Unexpected error: {e}")
        return ExtractedProfile(
            position="Unknown",
            estimated_grade="Junior",  # Безопасный выбор
            experience_summary="Insufficient data provided by candidate.",
            confidence=30
        )

In [ ]:
def extract_profile_fallback(intro_text: str) -> ExtractedProfile:
    """
    Fallback метод для извлечения профиля при ошибках LLM
    """
    position = "Unknown"
    grade = "Junior"
    experience = intro_text[:200] + "..." if len(intro_text) > 200 else intro_text

    # Простая эвристика для определения позиции
    keywords = {
        "developer": "Software Developer",
        "engineer": "Software Engineer",
        "analyst": "Data Analyst",
        "scientist": "Data Scientist",
        "manager": "Project Manager",
        "designer": "UX Designer"
    }

    intro_lower = intro_text.lower()
    for key, value in keywords.items():
        if key in intro_lower:
            position = value
            break

    # Эвристика для определения грейда
    if "senior" in intro_lower:
        grade = "Senior"
    elif "junior" in intro_lower:
        grade = "Junior"
    elif "mid" in intro_lower or "middle" in intro_lower:
        grade = "Middle"

    return ExtractedProfile(
        position=position,
        estimated_grade=grade,
        experience_summary=experience,
        confidence=40
    )

In [ ]:
def observer_agent(state: InterviewState) -> InterviewState:

    if not state.last_user_message:
        return state


    # ===== PROMPT =====

    prompt = f"""
Ты — аналитический модуль интервью.

Контекст:

Профиль кандидата:
Позиция: {state.candidate_profile.position}
Уровень: {state.candidate_profile.grade}
Опыт: {state.candidate_profile.experience}

Последний вопрос интервьюера:
"{state.agent_visible_message}"

Ответ кандидата:
"{state.last_user_message}"

Сложность: {state.difficulty_level}

Задача:

Проанализируй сообщение кандидата.

Определи:

1. Является ли сообщение вопросом к интервьюеру.
2. Соответствует ли ответ заданному вопросу.
3. Есть ли выдумка, дезинформация или уход от темы.

Верни JSON строго в следующем формате:

{{
  "is_question": true | false,
  "evaluation": "strong" | "ok" | "weak" | "hallucination",
  "instruction": "answer_candidate" | "challenge" | "proceed" | "stop",
  "internal_thoughts": "краткое описание оценки"
}}

Правила:

- Если кандидат задал вопрос → is_question = true и instruction = "answer_candidate"
- Если есть выдумка → evaluation = "hallucination" и instruction = "challenge"
- Если ответ не по теме → evaluation = "weak" и instruction = "challenge"
- Если ответ корректный → evaluation = "ok" и instruction = "proceed"
- Никакого текста вне JSON
"""

    # ===== LLM CALL =====

    try:

        response = observer_llm.invoke([
            SystemMessage(content=OBSERVER_SYSTEM_PROMPT),
            HumanMessage(content=prompt)
        ])

        raw = (response.content or "").strip()

        raw = re.sub(r"```json|```", "", raw).strip()

        match = re.search(r"\{.*\}", raw, re.S)

        if not match:
            raise ValueError("No JSON found")

        result = json.loads(match.group(0))

    except Exception as e:

        # Fallback если LLM сломался

        result = {
            "is_question": False,
            "evaluation": "weak",
            "instruction": "proceed",
            "internal_thoughts": "Fallback: ошибка анализа observer"
        }

    # ===== VALIDATION =====

    is_question = result.get("is_question", False)

    if not isinstance(is_question, bool):
        is_question = False

    evaluation = result.get("evaluation", "ok")

    if evaluation not in {"strong", "ok", "weak", "hallucination"}:
        evaluation = "ok"

    instruction = result.get("instruction", "proceed")

    allowed = {
        "answer_candidate",
        "challenge",
        "proceed",
        "stop"
    }

    if instruction not in allowed:
        instruction = "proceed"

    internal_thoughts = result.get(
        "internal_thoughts",
        "Нет комментариев"
    )

    # ===== BUSINESS LOGIC =====

    # Вопрос кандидата всегда в приоритете
    if is_question:
        instruction = "answer_candidate"

    # Галлюцинация = красный флаг
    if evaluation == "hallucination":
        state.red_flag = True
        state.difficulty_adjustment = "down"

    elif instruction == "challenge":
        state.difficulty_adjustment = "up"


    elif instruction == "proceed":
        state.difficulty_adjustment = None

    # ===== LOGGING =====

    state.observer_notes = f"{internal_thoughts} | {evaluation}"
    state.next_action = instruction
    state.observer_instruction = instruction
    state.observer_evaluation = evaluation

    return state

In [ ]:
def interviewer_agent(state: InterviewState) -> InterviewState:


    # ===== ОТВЕТ НА ВОПРОС КАНДИДАТА =====
    if state.next_action == "answer_candidate":

        prompt = f"""
Кандидат задал вопрос:

"{state.last_user_message}"

Ответь кратко и профессионально.
Затем верни разговор к техническому интервью.
"""

        response = interviewer_llm.invoke([
            SystemMessage(content=INTERVIEWER_SYSTEM_PROMPT),
            HumanMessage(content=prompt)
        ])

        state.agent_visible_message = response.content.strip()

        # Мысли интервьюера
        state.interviewer_notes = (
            "Ответил на вопрос кандидата и вернул фокус на интервью."
        )

        state.next_action = "proceed"

        return state


    # ===== РЕАКЦИЯ НА ГАЛЛЮЦИНАЦИИ =====
    if state.red_flag:

        prompt = f"""
Кандидат дал ложную или вымышленную информацию:

"{state.last_user_message}"

Вежливо укажи на ошибку.
Попроси объяснить корректно.
"""

        response = interviewer_llm.invoke([
            SystemMessage(content=INTERVIEWER_SYSTEM_PROMPT),
            HumanMessage(content=prompt)
        ])

        state.agent_visible_message = response.content.strip()

        state.interviewer_notes = (
            "Обнаружена возможная галлюцинация — запросил уточнение."
        )

        state.red_flag = False

        return state


    # ===== УПРАВЛЕНИЕ СЛОЖНОСТЬЮ =====
    if state.difficulty_adjustment == "up":
        state.difficulty_level = "hard"
        diff_note = "Усложнил следующий вопрос."

    elif state.difficulty_adjustment == "down":
        state.difficulty_level = "easy"
        diff_note = "Упростил следующий вопрос."

    else:
        diff_note = "Сохранил текущий уровень сложности."


    # ===== ОСНОВНОЙ ВОПРОС =====
    prompt = f"""
Контекст интервью:

Позиция: {state.candidate_profile.position}
Уровень: {state.candidate_profile.grade}
Опыт: {state.candidate_profile.experience}
Сложность: {state.difficulty_level}

Пройденные темы: {state.covered_topics}

Последний вопрос: {state.agent_visible_message}
Последний ответ: {state.last_user_message}

Инструкция:
{state.next_action}

Задай следующий технический вопрос.

Правила:
- Один вопрос
- Без английского
- Не повторяйся
"""

    response = interviewer_llm.invoke([
        SystemMessage(content=INTERVIEWER_SYSTEM_PROMPT),
        HumanMessage(content=prompt)
    ])

    message = response.content.strip()

    if not message:
        message = "Поясните, пожалуйста, свой ответ подробнее."

    state.agent_visible_message = message


    # Мысли интервьюера
    state.interviewer_notes = (
        f"Сформировал следующий вопрос. {diff_note}"
    )


    if state.next_action == "stop":
        state.stop_interview = True
        return state


    state.current_topic = message
    state.covered_topics.append(message)

    return state


In [ ]:
def log_turn(state: InterviewState) -> InterviewState:

    thoughts = ""

    if state.observer_notes:
        thoughts += f"[Observer]: {state.observer_notes}\n"

    if state.interviewer_notes:
        thoughts += f"[Interviewer]: {state.interviewer_notes}\n"


    state.dialogue_history.append(
        TurnLog(
            turn_id=state.turn_id,
            agent_visible_message=state.agent_visible_message or "",
            user_message=state.last_user_message or "",
            internal_thoughts=thoughts.strip()
        )
    )

    # Сброс после записи
    state.observer_notes = None
    state.interviewer_notes = None

    state.turn_id += 1

    return state

In [ ]:
def generate_feedback(state: InterviewState) -> dict:
    """
    Generates final interview feedback using Hiring Manager LLM
    with observer-based scoring and intelligent fallback
    with fully dynamic roadmap
    """

    conversation = []

    # ===== Сбор истории =====
    for turn in state.dialogue_history:

        eval_tag = "unknown"

        if turn.internal_thoughts:
            text = turn.internal_thoughts.lower()

            if "hallucination" in text:
                eval_tag = "hallucination"
            elif "strong" in text:
                eval_tag = "strong"
            elif "weak" in text:
                eval_tag = "weak"
            elif "ok" in text:
                eval_tag = "ok"

        conversation.append({
            "turn_id": turn.turn_id,
            "question": turn.agent_visible_message,
            "answer": turn.user_message,
            "evaluation": eval_tag,
            "observer_notes": turn.internal_thoughts
        })


    # ===== Статистика =====
    stats = {
        "strong": 0,
        "ok": 0,
        "weak": 0,
        "hallucination": 0,
        "unknown": 0
    }

    for turn in conversation:
        tag = turn.get("evaluation", "unknown")
        stats[tag] = stats.get(tag, 0) + 1


    # ===== PROMPT =====
    prompt = f"""
Ты — опытный hiring manager.

Тебе передана полная история интервью и оценки observer.

Профиль кандидата:
Имя: {state.candidate_profile.name}
Позиция: {state.candidate_profile.position}
Уровень: {state.candidate_profile.grade}
Опыт: {state.candidate_profile.experience}

Статистика ответов:
{json.dumps(stats, ensure_ascii=False, indent=2)}

История интервью:
{json.dumps(conversation, ensure_ascii=False, indent=2)}

Твоя задача:

1. Оцени кандидата честно.
2. Выяви реальные сильные и слабые стороны.
3. Сформируй персональный план развития (roadmap),
   основанный ТОЛЬКО на этом интервью.

Roadmap должен:
- Быть конкретным
- Отражать пробелы знаний
- Не быть шаблонным
- Быть связан с ошибками кандидата

Обязательно заполни ВСЕ разделы.

Верни ТОЛЬКО валидный JSON строго по формату.

Никаких пояснений.
Никакого markdown.
"""


    # ===== LLM CALL =====
    try:

        response = hiring_llm.invoke([
            SystemMessage(content=HIRING_MANAGER_PROMPT),
            HumanMessage(content=prompt)
        ])

        raw = (response.content or "").strip()

        raw = re.sub(r"```json|```", "", raw).strip()

        if not raw:
            raise ValueError("Empty LLM response")

        parsed = json.loads(raw)

        # Валидация roadmap
        if "roadmap" not in parsed or not parsed["roadmap"]:
            raise ValueError("Missing roadmap")

        if not isinstance(parsed["roadmap"], list):
            raise ValueError("Invalid roadmap format")

        return parsed


    # ===== INTELLIGENT FALLBACK =====
    except Exception:

        # Анализ слабых мест из диалога
        weak_topics = []

        for turn in conversation:

            if turn["evaluation"] in {"weak", "hallucination"}:

                q = turn["question"][:80]

                weak_topics.append(q)


        # Уникальные
        weak_topics = list(set(weak_topics))


        roadmap = []


        for topic in weak_topics:

            roadmap.append(
                f"Повторить и отработать тему: «{topic}»"
            )


        # Если совсем пусто — общий план
        if not roadmap:

            roadmap = [
                "Углубить знание базовых конструкций Python",
                "Практиковаться в объяснении алгоритмов",
                "Улучшить навыки анализа задач",
                "Решать задачи уровня своего грейда"
            ]


        # Оценка confidence
        hallucinations = stats.get("hallucination", 0)
        weak = stats.get("weak", 0)
        strong = stats.get("strong", 0)

        confidence = 60
        confidence -= hallucinations * 15
        confidence -= weak * 5
        confidence += strong * 5

        confidence = max(20, min(90, confidence))


        # Recommendation
        if hallucinations > 0 or weak >= 3:
            recommendation = "No Hire"
        elif strong >= 3:
            recommendation = "Strong Hire"
        else:
            recommendation = "Hire"


        # Soft skills
        clarity = "medium"
        honesty = "medium"
        engagement = "medium"

        if weak >= 2:
            clarity = "low"

        if hallucinations > 0:
            honesty = "low"

        if stats.get("unknown", 0) > 2:
            engagement = "low"


        return {
            "decision": {
                "grade": state.candidate_profile.grade,
                "hiring_recommendation": recommendation,
                "confidence_score": confidence
            },

            "hard_skills": {
                "confirmed_skills": [],
                "knowledge_gaps": []
            },

            "soft_skills": {
                "clarity": clarity,
                "honesty": honesty,
                "engagement": engagement
            },

            "roadmap": roadmap
        }

In [ ]:
def format_feedback(feedback_json: dict) -> str:
    """
    Converts feedback JSON to readable text format
    """

    decision = feedback_json.get("decision", {})
    hard = feedback_json.get("hard_skills", {})
    soft = feedback_json.get("soft_skills", {})
    roadmap = feedback_json.get("roadmap", [])

    text = []

    # Вердикт
    text.append("=== Вердикт ===")
    text.append(f"Grade: {decision.get('grade')}")
    text.append(f"Recommendation: {decision.get('hiring_recommendation')}")
    text.append(f"Confidence: {decision.get('confidence_score')}%")

    # Hard skills
    text.append("\n=== Анализ Hard Skills ===")

    text.append("\nПодтверждённые навыки:")
    for s in hard.get("confirmed_skills", []):
        text.append(f"- {s.get('topic')}: {s.get('evidence')}")

    text.append("\nПробелы в знаниях:")
    for g in hard.get("knowledge_gaps", []):
        text.append(f"- {g.get('topic')}")
        text.append(f"  Ответ: {g.get('candidate_answer')}")
        text.append(f"  Правильно: {g.get('correct_answer')}")

    # Soft skills
    text.append("\n=== Soft Skills ===")
    text.append(f"Clarity: {soft.get('clarity')}")
    text.append(f"Honesty: {soft.get('honesty')}")
    text.append(f"Engagement: {soft.get('engagement')}")

    # Roadmap
    text.append("\n=== Roadmap ===")

    for r in roadmap:
        text.append(f"- {r}")

    return "\n".join(text)

In [ ]:
def main():

    # ===== ВВОД ФИО =====
    participant_name = input("Введите ваше ФИО: ").strip()

    if not participant_name:
        participant_name = "Не указано"

    print("\nРасскажи о себе: опыт, технологии, на какую позицию метишь.")
    intro = input("Candidate: ")

    # ===== АНАЛИЗ ПРОФИЛЯ =====
    profile = extract_candidate_profile(intro, profile_llm)

    state = InterviewState(
        candidate_profile=CandidateProfile(
            name=participant_name,   # ← сохраняем ФИО
            position=profile.position,
            grade=profile.estimated_grade,
            experience=profile.experience_summary
        )
    )

    # ===== НАСТРОЙКА СЛОЖНОСТИ =====
    GRADE_TO_DIFFICULTY = {
        "Junior": "easy",
        "Middle": "medium",
        "Senior": "hard"
    }

    state.difficulty_level = GRADE_TO_DIFFICULTY.get(
        profile.estimated_grade,
        "easy"
    )

    state.next_action = "ask_new"

    # ===== ПЕРВЫЙ ВОПРОС =====
    state = interviewer_agent(state)

    print("\nInterviewer:", state.agent_visible_message)

    max_turns = 11
    turn_count = 1

    # ===== ОСНОВНОЙ ЦИКЛ =====
    while not state.stop_interview and turn_count < max_turns:

        # ВВОД ОТ КАНДИДАТА
        user_input = input("\nCandidate: ").strip()

        if user_input.lower().startswith("стоп"):
            state.stop_interview = True
            break

        state.last_user_message = user_input

        # ОБРАБОТКА АГЕНТАМИ
        state = observer_agent(state)
        state = interviewer_agent(state)
        state = log_turn(state)

        # ВЫВОД
        print("\nInterviewer:", state.agent_visible_message)

        turn_count += 1

    print("\n=== Interview finished ===")

    # ===== ФИДБЭК =====
    feedback = generate_feedback(state)

    formatted_feedback = format_feedback(feedback)

    print("\n" + "=" * 50)
    print("FINAL FEEDBACK:")
    print("=" * 50)
    print(formatted_feedback)

    # ===== СОХРАНЕНИЕ ЛОГА =====

    filename = f"interview_log_{participant_name.replace(' ', '_')}.json"

    with open(filename, "w", encoding="utf-8") as f:

        json.dump(
            {
                "participant_name": state.candidate_profile.name,

                "turns": [
                    {
                        "turn_id": t.turn_id,
                        "agent_visible_message": t.agent_visible_message,
                        "user_message": t.user_message,
                        "internal_thoughts": t.internal_thoughts
                    }
                    for t in state.dialogue_history
                ],

                # В ТЗ — строка, не объект
                "final_feedback": formatted_feedback
            },
            f,
            ensure_ascii=False,
            indent=2
        )

    print(f"\nЛог сохранён в файл: {filename}")

In [ ]:
main()

Введите ваше ФИО: Коченов Илья Сергеевич

Расскажи о себе: опыт, технологии, на какую позицию метишь.
Candidate: ривет, я Иван. Я ручной тестировщик. Сразу предупреждаю: программировать я не умею, автоматизацию не знаю.

Interviewer: Отлично, давайте начнём. Мой первый вопрос:  

**Что такое тест-кейс и перечислите основные элементы, которые обычно в него входят?**

Candidate: Тест-кейс — это документ, в котором описан конкретный сценарий проверки работы определённой функции или части системы. Он нужен для того, чтобы тестировщик понимал, что именно и как нужно проверить. Основная цель тест-кейса — убедиться, что программа работает в соответствии с требованиями. Каждый тест-кейс должен быть понятным, однозначным и воспроизводимым.  В первую очередь в тест-кейсе указывается его уникальный идентификатор, чтобы его можно было легко найти в системе тестирования. Далее обычно прописывается название или краткое описание проверки. Важным элементом являются предусловия — условия, которые должн